# Correlation Analysis
*by XY, 2023*
*Master "Applied Data Science" @ Nordakademie*
*Modul: Text Analytics*

### General pre-work

In [2]:
# Import necessary libraries

# Overall needed
import pandas as pd # to work with data frames
from scipy.stats import chi2_contingency

# Needed for visualisations and smaller functions such as time tracking or saving files
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import os # to define a dedicated output folder

In [3]:
output_folder = 'data/analysis' # refer to a new folder to store only sentiment result files

# check if folder can be found, else create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Add the normal distribution stock values to the combined sentiment table

csv_file_path_normal_3 = 'data/transfer/cleaned_articles_normalverteilt_3.csv' # define the path of the input csv file with 3 categories, normal distribution

# Load the data set
df_3categories_normal = pd.read_csv(csv_file_path_normal_3, encoding='utf-8', quoting=csv.QUOTE_ALL) # ensuring the right encoding as in the csv file we still encounter incorrectly encoded special characters

csv_file_path_combined_sentiments = 'data/sentiment_results/sentiment_results_combined.csv' # define the path of the input csv file with the sentiment results

# Load the data set
df_combined_sentiments = pd.read_csv(csv_file_path_combined_sentiments, encoding='utf-8', quoting=csv.QUOTE_ALL) # ensuring the right encoding as in the csv file we still encounter incorrectly encoded special characters

## Pre-Processing

In [4]:
# Merge the two data frames based on a common column, such as an index or a specific column name
# In this example, I assume you want to use the index as the merge key
df_merged_3_categories = df_combined_sentiments.merge(df_3categories_normal[['Stock_ValueStd']], left_index=True, right_index=True)

# Save the merged data frame to a new variable or file if needed
# merged_df contains the combined data with the "Stock_ValueStd" column added

# If you want to save the merged data frame to a new CSV file, you can use the following:
# merged_df.to_csv('merged_data.csv', index=False)

print(df_merged_3_categories.head(3))

   Row_Number                   Unternehmen Newstyp   Quelle Nearest_Date  \
0           1  Porsche Automobil Holding SE    News  onvista   2021-06-01   
1           2                    Beiersdorf    News  onvista   2021-06-02   
2           3          Heidelberg Materials    News  onvista   2021-06-02   

                                        Cleaned_Text Stock_Value  \
0  neu Rumor Porschebörsengang Sixt Berenberg stu...     neutral   
1  Beiersdorf Aktie Kaufempfehlung beflügeln Bere...    positive   
2  Heidelbergcement klimaneutral Zementwerk Weg B...    positive   

   TextBlob_Sentiment_Score TextBlob_Evenly_Separated_Label  \
0                   -0.0500                        Negative   
1                    0.2000                        Positive   
2                   -0.0875                        Negative   

  TextBlob_Normal_Distribution_Label  NLTK_Sentiment  \
0                            Neutral         -0.3612   
1                           Positive          0.0000 

In [5]:
#Rename and reposition columns

# 1. Rename column "Stock_Value" to "Stock_Value_evenly_separated"
df_merged_3_categories.rename(columns={'Stock_Value': 'Stock_Value_Evenly_Separated'}, inplace=True)

# 2. Rename column "Stock_ValueStd" to "Stock_Value_normal_distribution"
df_merged_3_categories.rename(columns={'Stock_ValueStd': 'Stock_Value_Normal_Distribution'}, inplace=True)

# 3. Reorder columns to move "Stock_Value_normal_distribution" to the 8th column
cols = df_merged_3_categories.columns.tolist()
cols.insert(7, cols.pop(cols.index('Stock_Value_Normal_Distribution')))
df_merged_3_categories = df_merged_3_categories[cols]

# 4. Capitalize the stock value
df_merged_3_categories['Stock_Value_Evenly_Separated'] = df_merged_3_categories['Stock_Value_Evenly_Separated'].str.capitalize()
df_merged_3_categories['Stock_Value_Normal_Distribution'] = df_merged_3_categories['Stock_Value_Normal_Distribution'].str.capitalize()

In [ ]:
# Save the merged file in the correct repository folder

# Save to an Excel file
excel_output_file_analysis_3 = os.path.join(output_folder, 'analysis_3_categories.xlsx')
df_merged_3_categories.to_excel(excel_output_file_analysis_3, index=False)

# Save the results to a csv file with the same name
csv_output_file_analysis_3 = os.path.join(output_folder, 'analysis_3_categories.csv')
df_merged_3_categories.to_csv(csv_output_file_analysis_3, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)

# Print final info
print(f"Results saved as {csv_output_file_analysis_3} and {excel_output_file_analysis_3}.")

## Correlation Analysis

### 3 Categories

#### TextBlob Even Separation

In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df_merged_3_categories['Stock_Value_Evenly_Separated'], df_merged_3_categories['TextBlob_Evenly_Separated_Label'])

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Output the chi-squared statistic and p-value
print(f"Chi-Squared Statistic: {chi2}")
print(f"P-Value: {p}")

# Interpret the results
alpha = 0.05  # Significance level
if p <= alpha:
    print("There is a significant association between the two categorical variables Stock_Value_Evenly_Separated and TextBlob_Evenly_Separated_Label.")
else:
    print("There is no significant association between the two categorical variables Stock_Value_Evenly_Separated and TextBlob_Evenly_Separated_Label.")

##### Cramer's V (or Cramer's Phi) and the Phi Coefficient (φ), analysing strength of relationship

In [ ]:
# Calculate Cramer's V
n = contingency_table.sum().sum()
min_dim = min(contingency_table.shape) - 1
cramer_v = (chi2 / n) ** 0.5 / min_dim

# Calculate the Phi Coefficient (φ)
phi_coefficient = (chi2 / n) ** 0.5

print(f"Cramer's V: {cramer_v}")
print(f"Phi Coefficient (φ): {phi_coefficient}")

#### Additional info about results
*Modul: Text Analytics*

Yes, it is **possible for Cramer's V (or Cramer's Phi) and the Phi Coefficient (φ) to be close to 0 while the chi-square test identifies a significant relationship**. This can occur when there is a statistically significant association between the variables, but the strength of that association is weak.

The chi-square test primarily assesses the statistical significance of the association, meaning it tells you whether there is evidence that the two categorical variables are related. However, it doesn't provide information about the strength or the practical significance of that relationship.

Cramer's V and the Phi Coefficient provide measures of effect size, indicating how strong the relationship is. A small effect size, reflected in values close to 0, suggests a weak association, while a large effect size indicates a strong association.

So, it's possible for the chi-square test to detect a statistically significant relationship even if the association is weak, leading to low values for Cramer's V and Phi Coefficient. The significance test assesses whether there is any association, while the effect size measures how substantial that association is in practical terms.

In [ ]:
# Define the order for the categories
order = ['Negative', 'Neutral', 'Positive']

# Create bar charts for each categorical variable with the specified order
sns.countplot(data=df_merged_3_categories, x='Stock_Value_Evenly_Separated', order=order)
plt.title('Distribution of Stock Value Categories')
plt.show()

sns.countplot(data=df_merged_3_categories, x='TextBlob_Evenly_Separated_Label', order=order)
plt.title('Distribution of TextBlob Sentiment Label Categories')
plt.show()

In [ ]:
contingency_table = pd.crosstab(df_merged_3_categories['Stock_Value_Evenly_Separated'], df_merged_3_categories['TextBlob_Evenly_Separated_Label'])

contingency_table.plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Stock Value Categories vs. TextBlob Sentiment Label Categories')
plt.show()

In [ ]:
contingency_table.plot(kind='bar')
plt.title('Clustered Bar Chart of Stock Value Categories vs. TextBlob Sentiment Label Categories')
plt.show()

In [ ]:
# Create a contingency table to count the co-occurrence of values
contingency_table = pd.crosstab(df_merged_3_categories['Stock_Value_Evenly_Separated'], df_merged_3_categories['TextBlob_Evenly_Separated_Label'])

# Define the order of categories
category_order = ['Negative', 'Neutral', 'Positive']

# Create a stacked bar chart
ax = contingency_table[category_order].plot(kind='bar', stacked=True, figsize=(8, 6,))

# Add labels and a legend
ax.set_xlabel('Stock Value')
ax.set_ylabel('Count')
ax.set_title('Relationship Between Stock Value and TextBlob Sentiment Label')
ax.legend(title='Sentiment Label', title_fontsize=12)

plt.xticks(rotation=0)  # Keep x-axis labels horizontal

plt.show()

In [ ]:
# Create a DataFrame to store the counts
counts_df = pd.DataFrame(index=['Negative', 'Neutral', 'Positive'], columns=['Stock_Value_Evenly_Separated', 'TextBlob_Evenly_Separated_Label'])

# Count how often each category appears for "Stock_Value_Evenly_Separated"
counts_df['Stock_Value_Evenly_Separated'] = df_merged_3_categories['Stock_Value_Evenly_Separated'].value_counts()

# Count how often both "Stock_Value_Evenly_Separated" and "TextBlob_Evenly_Separated_Label" are the same
for category in ['Negative', 'Neutral', 'Positive']:
    counts_df.loc[category, 'TextBlob_Evenly_Separated_Label'] = ((df_merged_3_categories['Stock_Value_Evenly_Separated'] == category) & (df_merged_3_categories['TextBlob_Evenly_Separated_Label'] == category)).sum()

# Print the counts
print(counts_df)